## Machine Translation Project

### Introduction
In this project, I have worked on an end-to-end machine translation pipeline which accepts text in one language (input) and returns its translation in another languague (output). This pipeline specifically accpets text in English language as input and outputs its French translation. 

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


## Dataset

I have used a dataset (contains small vocabulary) that is creadted by Udaticy. You can find that in my Github repo. Here is the link to most common dataset used for machine translation projects -- [WMT](http://www.statmt.org/). Since it's quite big this will take a long time to train a neural network. In future I'll try to use it to train my models.

### Load Data

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


## Pre-processing

### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


## Preprocess
We need to convert our textual data into numbers, so that we can feed it into our neural network model. We will be using the following preprocess methods to do that;
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenization
We can turn each character into a number or each word into a number. These are called character and word ids, respectively. Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

We will use Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function to tokenize `english_sentences` and `french_sentences`.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding
When batching the sequence of word ids together, each sequence needs to be the same length. Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

All the English sequences and the French sequences should have the same length; in order to do that we will be padding the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = len(max(x, key=len))

    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models

Experimenting with various neural network architectures. I'll be training these four simple architectures:
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is a Encoder-Decoder RNN
- Model 5 is a Bidirectional RNN with Embedding

### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.

In [10]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: simple RNN 
A basic RNN model is a good baseline for sequence data.

In [11]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)
    
    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 7s 65us/step - loss: 3.5259 - acc: 0.4194 - val_loss: nan - val_acc: 0.4671
Epoch 2/10
110288/110288 [==============================] - 6s 59us/step - loss: 2.4520 - acc: 0.4747 - val_loss: nan - val_acc: 0.4852
Epoch 3/10
110288/110288 [==============================] - 6s 58us/step - loss: 2.1752 - acc: 0.5116 - val_loss: nan - val_acc: 0.5315
Epoch 4/10
110288/110288 [==============================] - 6s 58us/step - loss: 1.9139 - acc: 0.5555 - val_loss: nan - val_acc: 0.5709
Epoch 5/10
110288/110288 [==============================] - 6s 58us/step - loss: 1.7472 - acc: 0.5812 - val_loss: nan - val_acc: 0.5863
Epoch 6/10
110288/110288 [==============================] - 6s 58us/step - loss: 1.6510 - acc: 0.5923 - val_loss: nan - val_acc: 0.5982
Epoch 7/10
110288/110288 [==============================] - 6s 58us/step - loss: 1.5827 - acc: 0.6005 - val_loss: nan - val_acc: 0.60

### Model 2: RNN with Embedding
Converting the words into ids is a good step to preprocess your input inorder to feed them to neural network model, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, I have built a simple RNN with Embeddings.

In [12]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    embed_layer = Embedding(english_vocab_size, 64, input_length=output_sequence_length)(input_seq)
    rnn = GRU(64, return_sequences=True)(embed_layer)

    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)
    
    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)

# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s 69us/step - loss: 3.8092 - acc: 0.3986 - val_loss: nan - val_acc: 0.4093
Epoch 2/10
110288/110288 [==============================] - 7s 66us/step - loss: 2.6829 - acc: 0.4465 - val_loss: nan - val_acc: 0.4982
Epoch 3/10
110288/110288 [==============================] - 7s 67us/step - loss: 1.9740 - acc: 0.5621 - val_loss: nan - val_acc: 0.6188
Epoch 4/10
110288/110288 [==============================] - 7s 66us/step - loss: 1.4174 - acc: 0.6593 - val_loss: nan - val_acc: 0.6982
Epoch 5/10
110288/110288 [==============================] - 7s 66us/step - loss: 1.1374 - acc: 0.7193 - val_loss: nan - val_acc: 0.7436
Epoch 6/10
110288/110288 [==============================] - 7s 67us/step - loss: 0.9535 - acc: 0.7604 - val_loss: nan - val_acc: 0.7777
Epoch 7/10
110288/110288 [==============================] - 7s 66us/step - loss: 0.8175 - acc: 0.7893 - val_loss: nan - val_acc: 0.80

### Model 3: Bidirectional RNNs
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in. They are able to see the future data. So, in this model I have experimented with a simple Bidirectional RNN.

In [13]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    bdrnn = Bidirectional(GRU(64, return_sequences=True))(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(bdrnn)
    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_bd_model(bd_model)

# Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

bdrnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bdrnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(bdrnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 10s 94us/step - loss: 3.1746 - acc: 0.4615 - val_loss: nan - val_acc: 0.5218
Epoch 2/10
110288/110288 [==============================] - 10s 90us/step - loss: 1.9048 - acc: 0.5624 - val_loss: nan - val_acc: 0.5782
Epoch 3/10
110288/110288 [==============================] - 10s 90us/step - loss: 1.6063 - acc: 0.5930 - val_loss: nan - val_acc: 0.6058
Epoch 4/10
110288/110288 [==============================] - 10s 90us/step - loss: 1.4667 - acc: 0.6098 - val_loss: nan - val_acc: 0.6169
Epoch 5/10
110288/110288 [==============================] - 10s 90us/step - loss: 1.3769 - acc: 0.6209 - val_loss: nan - val_acc: 0.6271
Epoch 6/10
110288/110288 [==============================] - 10s 90us/step - loss: 1.3140 - acc: 0.6344 - val_loss: nan - val_acc: 0.6395
Epoch 7/10
110288/110288 [==============================] - 10s 90us/step - loss: 1.2650 - acc: 0.6444 - val_loss: nan - val_ac

### Model 4: Encoder-Decoder RNN
Time to look at encoder-decoder models. This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

In [14]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    
    inputs = Input(input_shape[1:])
    encoder = LSTM(64)(inputs)
    decoder = RepeatVector(output_sequence_length)(encoder)
    decoder = LSTM(64, return_sequences=True)(decoder)
    decoded = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder)

    model = Model(inputs=inputs, outputs=decoded)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model

tests.test_encdec_model(encdec_model)


# Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))


encdec_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

encdec_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 13s 121us/step - loss: 3.5652 - acc: 0.4003 - val_loss: nan - val_acc: 0.4445
Epoch 2/10
110288/110288 [==============================] - 13s 115us/step - loss: 2.5876 - acc: 0.4648 - val_loss: nan - val_acc: 0.4947
Epoch 3/10
110288/110288 [==============================] - 13s 115us/step - loss: 2.3509 - acc: 0.4946 - val_loss: nan - val_acc: 0.5007
Epoch 4/10
110288/110288 [==============================] - 13s 115us/step - loss: 2.2253 - acc: 0.5089 - val_loss: nan - val_acc: 0.5179
Epoch 5/10
110288/110288 [==============================] - 13s 114us/step - loss: 2.1168 - acc: 0.5205 - val_loss: nan - val_acc: 0.5278
Epoch 6/10
110288/110288 [==============================] - 13s 114us/step - loss: 1.9570 - acc: 0.5375 - val_loss: nan - val_acc: 0.5456
Epoch 7/10
110288/110288 [==============================] - 13s 115us/step - loss: 1.8444 - acc: 0.5504 - val_loss: nan -

### Model 5: Bidirectional RNN with Embedding

In [15]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate=1e-3

    input_seq = Input(input_shape[1:])
    emb = Embedding(english_vocab_size, 64, input_length=output_sequence_length)(input_seq)
    bdrnn = Bidirectional(GRU(64, return_sequences=True))(emb)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(bdrnn)

    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_model_final(model_final)


print('Final Model Loaded')

# Pad the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Train
emb_bdrnn = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

emb_bdrnn.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(emb_bdrnn.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 12s 113us/step - loss: 3.4711 - acc: 0.4274 - val_loss: nan - val_acc: 0.4759
Epoch 2/10
110288/110288 [==============================] - 12s 104us/step - loss: 2.2956 - acc: 0.5085 - val_loss: nan - val_acc: 0.5663
Epoch 3/10
110288/110288 [==============================] - 12s 105us/step - loss: 1.5642 - acc: 0.6182 - val_loss: nan - val_acc: 0.6700
Epoch 4/10
110288/110288 [==============================] - 12s 106us/step - loss: 1.1366 - acc: 0.7110 - val_loss: nan - val_acc: 0.7422
Epoch 5/10
110288/110288 [==============================] - 12s 105us/step - loss: 0.8972 - acc: 0.7608 - val_loss: nan - val_acc: 0.7768
Epoch 6/10
110288/110288 [==============================] - 12s 106us/step - loss: 0.7465 - acc: 0.7930 - val_loss: nan - val_acc: 0.8061
Epoch 7/10
110288/110288 [==============================] - 12s 106us/step - loss: 0.6391 - acc: 0.818

## Prediction

In [16]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Train neural network using Model 5
    max_english_sequence_length = x.shape[1]
    max_french_sequence_length = y.shape[1]
    english_vocab_size = len(x_tk.word_index)
    french_vocab_size = len(y_tk.word_index)

    # Pad the input
    x = pad(x, max_french_sequence_length)

    # Train
    model = model_final(
        x.shape,
        max_french_sequence_length,
        english_vocab_size,
        french_vocab_size)

    model.fit(x, y, batch_size=1024, epochs=100, validation_split=0.2)
    
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/100
110288/110288 [==============================] - 12s 113us/step - loss: 3.4589 - acc: 0.4315 - val_loss: nan - val_acc: 0.4747
Epoch 2/100
110288/110288 [==============================] - 12s 105us/step - loss: 2.2411 - acc: 0.5148 - val_loss: nan - val_acc: 0.5767
Epoch 3/100
110288/110288 [==============================] - 12s 105us/step - loss: 1.5799 - acc: 0.6143 - val_loss: nan - val_acc: 0.6501
Epoch 4/100
110288/110288 [==============================] - 12s 106us/step - loss: 1.2367 - acc: 0.6829 - val_loss: nan - val_acc: 0.7115
Epoch 5/100
110288/110288 [==============================] - 12s 105us/step - loss: 0.9981 - acc: 0.7289 - val_loss: nan - val_acc: 0.7475
Epoch 6/100
110288/110288 [==============================] - 12s 105us/step - loss: 0.8286 - acc: 0.7642 - val_loss: nan - val_acc: 0.7804
Epoch 7/100
110288/110288 [==============================] - 12s 105us/step - loss: 0.6971 - acc: 0.7954 - val_loss